In [38]:
# Loading data

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
DF=pd.concat([train,test])

In [39]:
# Decoding names
def decodeTitle(name):
    signif_titles=['Mr.','Miss.','Mrs.','Master.'] 
    # cut exactly title from column Name
    title=name[name.find(',')+2:name.find('.')+1]
    if title in signif_titles:  return(title)
    else: return('Arist.')
    
DF['Title']=DF['Name'].apply(decodeTitle)

In [40]:
# Decodng cabins
def number(string):
    if pd.notnull(string):
        numbers=[int(i) for i in re.findall('(\d+)',string)]
        return(np.mean(numbers))
def symbol(string):
    if pd.notnull(string):
        return(True)
    else: return(False)

DF['CabinSymbol'] = DF.Cabin.apply(symbol)
DF['CabinNumber'] = DF.Cabin.apply(number)

C:\Users\user\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [41]:
Ages, Fares=[0],[0]
for i in [1,2,3]:
    Ages.append(DF[DF.Pclass==i]['Age'].mean())
    Fares.append(DF[DF.Pclass==i]['Fare'].mean())
def funca(row):
    if row[0]>0: 
        age=row[0]
    else: 
        age=Ages[row[7]]
    return(age)
def funcb(row):
    if row[3]>0: 
        fare=row[3]
    else: 
        fare=Fares[row[7]]
    return(fare)
                 
DF['Age']=DF.apply(funca,axis=1)
DF['Fare']=DF.apply(funcb,axis=1)

In [42]:
DF['Title']=DF['Title'].astype('category').cat.codes
DF['CabinSymbol']=DF['CabinSymbol'].astype('category').cat.codes
DF['Sex']=DF['Sex'].astype('category').cat.codes

In [43]:
from sklearn.utils import shuffle
DF = shuffle(DF, random_state = 187)

In [58]:
DF_train=DF[DF.Survived.notnull()]
DF_test=DF[DF.Survived.notnull()==False]
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

optimizer = GridSearchCV(RandomForestClassifier(), \
                         {'max_features' : [0.4, 0.6, 0.8, 1.0], 
                          'max_depth' : [5, 10, 15, 25, 35, 45, 55, 65], 
                          'n_estimators' : [1, 5, 10, 15, 30, 50, 70, 90, 100]},\
                         cv = 15, \
                         n_jobs=-1)
optimizer.fit(DF_train[['Age','Fare','SibSp','Parch','Pclass','Title','CabinSymbol','Sex']], DF_train['Survived'])
print('Random Forest Best Result', optimizer.best_score_)
print('Best Parameters', optimizer.best_params_)

Random Forest Best Result 0.8372615039281706
Best Parameters {'max_features': 0.6, 'n_estimators': 10, 'max_depth': 10}


In [59]:
def writeAnswer(filename, optimizer):
    prediction = optimizer.predict(np.hstack([DF_test[['Age','Fare','SibSp','Parch','Pclass','Title','CabinSymbol','Sex']]]))
    submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.to_csv(filename, index=False)
    
writeAnswer('RandomForestClassifier.csv', optimizer)

In [48]:
DF_train=DF[DF.Survived.notnull()].head(600)


891

In [53]:
DF_train=DF[DF.Survived.notnull()].tail(290)
prediction = optimizer.predict(np.hstack([DF_train[['Age','Fare','SibSp','Parch','Pclass','Title','CabinSymbol','Sex']]]))
submission = pd.DataFrame({
        "PassengerId": DF_train["Survived"],
        "Survived_2": prediction
        })


In [56]:
pd.crosstab(submission.PassengerId,submission.Survived_2)

Survived_2,0.0,1.0
PassengerId,,
0.0,158,19
1.0,30,83


In [57]:
(290-49)/290

0.8310344827586207